In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.compose import ColumnTransformer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\kille\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [2]:
stock_list = ['TSLA', 'MSFT', 'PG', 'META', 'AMZN']
stock_tweets_df = pd.read_csv('Resources/stock_tweets.csv')
stock_tweets_df = stock_tweets_df[stock_tweets_df['Stock Name'].isin(stock_list)]
stock_tweets_df = stock_tweets_df.drop(columns=['Company Name'])
stock_tweets_df.head()

,Date,Tweet,Stock Name
0,2022-09-29 23:41:16+00:00,Mainstream media has done an amazing job at br...,TSLA
1,2022-09-29 23:24:43+00:00,Tesla delivery estimates are at around 364k fr...,TSLA
2,2022-09-29 23:18:08+00:00,3/ Even if I include 63.0M unvested RSUs as of...,TSLA
3,2022-09-29 22:40:07+00:00,@RealDanODowd @WholeMarsBlog @Tesla Hahaha why...,TSLA
4,2022-09-29 22:27:05+00:00,"@RealDanODowd @Tesla Stop trying to kill kids,...",TSLA


In [3]:
stock_yfinance_df = pd.read_csv('Resources/stock_yfinance_data.csv')
stock_yfinance_df = stock_yfinance_df[stock_yfinance_df['Stock Name'].isin(stock_list)]
stock_yfinance_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Stock Name
0,2021-09-30,260.333344,263.043335,258.333344,258.493347,258.493347,53868000,TSLA
1,2021-10-01,259.466675,260.260010,254.529999,258.406677,258.406677,51094200,TSLA
2,2021-10-04,265.500000,268.989990,258.706665,260.510010,260.510010,91449900,TSLA
3,2021-10-05,261.600006,265.769989,258.066681,260.196655,260.196655,55297800,TSLA
4,2021-10-06,258.733337,262.220001,257.739990,260.916656,260.916656,43898400,TSLA


In [4]:
def get_date(date):
    return date[0:10]

In [5]:
stock_tweets_df['Datetime'] = stock_tweets_df['Date'].apply(get_date)
stock_tweets_df.head()

,Date,Tweet,Stock Name,Datetime
0,2022-09-29 23:41:16+00:00,Mainstream media has done an amazing job at br...,TSLA,2022-09-29
1,2022-09-29 23:24:43+00:00,Tesla delivery estimates are at around 364k fr...,TSLA,2022-09-29
2,2022-09-29 23:18:08+00:00,3/ Even if I include 63.0M unvested RSUs as of...,TSLA,2022-09-29
3,2022-09-29 22:40:07+00:00,@RealDanODowd @WholeMarsBlog @Tesla Hahaha why...,TSLA,2022-09-29
4,2022-09-29 22:27:05+00:00,"@RealDanODowd @Tesla Stop trying to kill kids,...",TSLA,2022-09-29


In [6]:
def get_score(tweet):
    analyzer = SentimentIntensityAnalyzer()
    score = analyzer.polarity_scores(tweet)['compound']
    return score

In [7]:
stock_tweets_df['Sentiment Score'] = stock_tweets_df['Tweet'].apply(get_score)

In [ ]:
date_stock_df = stock_tweets_df.groupby(['Datetime','Stock Name']).size().reset_index(name='Tweet Count')
date_stock_df

,Datetime,Stock Name,Tweet Count
0,2021-09-30,AMZN,5
1,2021-09-30,META,3
2,2021-09-30,MSFT,5
3,2021-09-30,PG,5
4,2021-09-30,TSLA,90
...,...,...,...
1763,2022-09-29,AMZN,10
1764,2022-09-29,META,1
1765,2022-09-29,MSFT,10
1766,2022-09-29,PG,10


In [ ]:
stock_tweets_df = pd.merge(stock_tweets_df,date_stock_df,
                           on=['Datetime','Stock Name'],
                           how='inner'
                           )
stock_tweets_df.head()

,Date,Tweet,Stock Name,Company Name,Datetime,Sentiment Score,Tweet Count
0,2022-09-29 23:41:16+00:00,Mainstream media has done an amazing job at br...,TSLA,"Tesla, Inc.",2022-09-29,0.0772,112
1,2022-09-29 23:24:43+00:00,Tesla delivery estimates are at around 364k fr...,TSLA,"Tesla, Inc.",2022-09-29,0.0000,112
2,2022-09-29 23:18:08+00:00,3/ Even if I include 63.0M unvested RSUs as of...,TSLA,"Tesla, Inc.",2022-09-29,0.2960,112
3,2022-09-29 22:40:07+00:00,@RealDanODowd @WholeMarsBlog @Tesla Hahaha why...,TSLA,"Tesla, Inc.",2022-09-29,-0.7568,112
4,2022-09-29 22:27:05+00:00,"@RealDanODowd @Tesla Stop trying to kill kids,...",TSLA,"Tesla, Inc.",2022-09-29,-0.8750,112


In [ ]:
stock_yfinance_df['Open/Close Diff'] = round(abs(stock_yfinance_df['Open'] - stock_yfinance_df['Close']),2)

In [ ]:
stock_yfinance_df['Prev Close Diff'] = stock_yfinance_df['Close'].diff()

In [ ]:
stock_yfinance_df = stock_yfinance_df.rename(columns={'Date':'Datetime'})

In [ ]:
stock_df = pd.merge(stock_tweets_df,stock_yfinance_df,on=['Datetime','Stock Name'])

In [ ]:
close_df = stock_df['Close']
stock_df = stock_df.drop(columns=['Date','Datetime','Tweet','Close'])

In [ ]:
categorical_df = stock_df.select_dtypes('object')
numerical_df = stock_df.select_dtypes(['int64','float64'])

In [ ]:
ct = ColumnTransformer([('scaler',StandardScaler(),numerical_df.columns)])
array = ct.fit_transform(numerical_df)
scaler = MinMaxScaler(feature_range=(0,1))
array = scaler.fit_transform(array)
numerical_df = pd.DataFrame(data=array,columns=ct.get_feature_names_out())

In [ ]:
categorical_df = pd.get_dummies(data=categorical_df,dtype=int)


In [ ]:
categorical_df

,Stock Name_AMZN,Stock Name_META,Stock Name_MSFT,Stock Name_PG,Stock Name_TSLA,"Company Name_Amazon.com, Inc.","Company Name_Meta Platforms, Inc.",Company Name_Microsoft Corporation,Company Name_Procter & Gamble Company,"Company Name_Tesla, Inc."
0,0,0,0,0,1,0,0,0,0,1
1,0,0,0,0,1,0,0,0,0,1
2,0,0,0,0,1,0,0,0,0,1
3,0,0,0,0,1,0,0,0,0,1
4,0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
42360,1,0,0,0,0,1,0,0,0,0
42361,1,0,0,0,0,1,0,0,0,0
42362,1,0,0,0,0,1,0,0,0,0
42363,1,0,0,0,0,1,0,0,0,0


In [ ]:
final_stock_df = pd.concat([categorical_df,numerical_df,close_df],axis=1)

In [ ]:
final_stock_df

,Stock Name_AMZN,Stock Name_META,Stock Name_MSFT,Stock Name_PG,Stock Name_TSLA,"Company Name_Amazon.com, Inc.","Company Name_Meta Platforms, Inc.",Company Name_Microsoft Corporation,Company Name_Procter & Gamble Company,"Company Name_Tesla, Inc.",scaler__Sentiment Score,scaler__Tweet Count,scaler__Open,scaler__High,scaler__Low,scaler__Adj Close,scaler__Volume,scaler__Open/Close Diff,scaler__Prev Close Diff,Close
0,0,0,0,0,1,0,0,0,0,1,0.539873,0.240781,0.583085,0.577800,0.540461,0.539232,0.274489,0.290826,0.253251,268.209991
1,0,0,0,0,1,0,0,0,0,1,0.500859,0.240781,0.583085,0.577800,0.540461,0.539232,0.274489,0.290826,0.253251,268.209991
2,0,0,0,0,1,0,0,0,0,1,0.650445,0.240781,0.583085,0.577800,0.540461,0.539232,0.274489,0.290826,0.253251,268.209991
3,0,0,0,0,1,0,0,0,0,1,0.118405,0.240781,0.583085,0.577800,0.540461,0.539232,0.274489,0.290826,0.253251,268.209991
4,0,0,0,0,1,0,0,0,0,1,0.058672,0.240781,0.583085,0.577800,0.540461,0.539232,0.274489,0.290826,0.253251,268.209991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42360,1,0,0,0,0,1,0,0,0,0,0.729028,0.008677,0.204230,0.199449,0.205119,0.201333,0.197219,0.030981,0.407088,164.251999
42361,1,0,0,0,0,1,0,0,0,0,0.242824,0.008677,0.204230,0.199449,0.205119,0.201333,0.197219,0.030981,0.407088,164.251999
42362,1,0,0,0,0,1,0,0,0,0,0.386497,0.008677,0.204230,0.199449,0.205119,0.201333,0.197219,0.030981,0.407088,164.251999
42363,1,0,0,0,0,1,0,0,0,0,0.974328,0.008677,0.204230,0.199449,0.205119,0.201333,0.197219,0.030981,0.407088,164.251999


In [ ]:
final_stock_df = final_stock_df.fillna(0.0)

In [ ]:
final_stock_df.to_csv('Resources/final_stock_data.csv',index=False)